# 4.5. Understanding the internals of NumPy to avoid unnecessary array copying

In [1]:
import numpy as np

In [2]:
def aid(x):
    # This function returns the memory
    # block address of an array.
    return x.__array_interface__['data'][0]

In [3]:
a = np.zeros(3)
aid(a), aid(a[1:])

(21535472, 21535480)

In [4]:
def get_data_base(arr):
    """For a given NumPy array, find the base array
    that owns the actual data."""
    base = arr
    while isinstance(base.base, np.ndarray):
        base = base.base
    return base


def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

In [5]:
print(arrays_share_data(a, a.copy()))

False


In [6]:
print(arrays_share_data(a, a[:1]))

True


In [7]:
import numpy as np
a = np.zeros(10)
ax = aid(a)
ax

32250112

In [8]:
b = a.copy()
aid(b) == ax

False

In [9]:
a *= 2
aid(a) == ax

True

In [10]:
c = a * 2
aid(c) == ax

False

In [11]:
%%timeit a = np.zeros(10000000)
a *= 2

4.85 ms ± 24 µs per loop (mean ± std. dev. of 7 runs,
100 loops each)


In [12]:
%%timeit a = np.zeros(10000000)
b = a * 2

7.7 ms ± 105 µs per loop (mean ± std. dev. of 7 runs,
100 loops each)


In [13]:
a = np.zeros((100, 100))
ax = aid(a)

In [14]:
b = a.reshape((1, -1))
aid(b) == ax

True

In [15]:
c = a.T.reshape((1, -1))
aid(c) == ax

False

In [16]:
%timeit b = a.reshape((1, -1))

330 ns ± 0.517 ns per loop (mean ± std. dev. of 7 runs
    1000000 loops each)


In [17]:
%timeit a.T.reshape((1, -1))

5 µs ± 5.68 ns per loop (mean ± std. dev. of 7 runs,
    100000 loops each)


In [18]:
d = a.flatten()
aid(d) == ax

False

In [19]:
e = a.ravel()
aid(e) == ax

True

In [20]:
%timeit a.flatten()

2.3 µs ± 18.1 ns per loop (mean ± std. dev. of 7 runs,
100000 loops each)


In [21]:
%timeit a.ravel()

199 ns ± 5.02 ns per loop (mean ± std. dev. of 7 runs,
10000000 loops each)


In [22]:
n = 1000

In [23]:
a = np.arange(n)
ac = a[:, np.newaxis]  # column vector
ar = a[np.newaxis, :]  # row vector

In [24]:
%timeit np.tile(ac, (1, n)) * np.tile(ar, (n, 1))

5.7 ms ± 42.6 µs per loop (mean ± std. dev. of 7 runs,
100 loops each)


In [25]:
%timeit ar * ac

784 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs,
1000 loops each)


```
offset = array.strides[0] * i1 + array.strides[1] * i2
```

In [26]:
a = np.random.rand(5000, 5000)

In [27]:
%timeit a[0, :].sum()

2.91 µs ± 20 ns per loop (mean ± std. dev. of 7 runs,
    100000 loops each)


In [28]:
%timeit a[:, 0].sum()

33.7 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs
    10000 loops each)
